## **Notebook #13**
## A proposal for hyperparameter tuning in learning models.
**Professor:** Fernando J. Von Zuben <br>
**Aluno(a):** Caio Francisco Garcia de Lima **RA** 195210


In [ ]:
import tensorflow as tf
import os
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=5)
model.summary()
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(x_test, y_test)
print("test loss, test acc:", results)
model_json = model.to_json()
json_file = open("model_MLP.json", "w")
json_file.write(model_json)
json_file.close()
model.save_weights("model_MLP.h5")
print("Model saved to disk")
os.getcwd()

11501568/11490434 [==============================] - 0s 0us/step
Epoch 1/5
1875/1875 [==============================] - 17s 8ms/step - loss: 0.2329 - accuracy: 0.9316
Epoch 2/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.1070 - accuracy: 0.9678
Epoch 3/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0784 - accuracy: 0.9762
Epoch 4/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0628 - accuracy: 0.9804
Epoch 5/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0552 - accuracy: 0.9828
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (32, 784)                 0         
                                                                 
 dense (Dense)               (32, 512)                 401920    
                                                                 
 dropout (Dr

'/content'

### Grid search for some hyperparameters.

In [ ]:
n_hidden_neurons = [256, 1024]
dropout_rate = [0.1, 0.5]
optimizers = ['adam','SGD']
epochs = [10, 300]
dic = {'n_hidden_neurons':[],'dropout_rate':[],'optimizers':[],'epochs':[], 'results':[]}

for hidden_neurons in n_hidden_neurons:
    for dropout in dropout_rate:
        for optimizer in optimizers:
            for epoch in epochs:
                model = tf.keras.models.Sequential([
                  tf.keras.layers.Flatten(),
                  tf.keras.layers.Dense(hidden_neurons, activation=tf.nn.relu),
                  tf.keras.layers.Dropout(dropout),
                  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
                ])
                model.compile(optimizer=optimizer,
                              loss='sparse_categorical_crossentropy',
                              metrics=['accuracy'])
                model.fit(x_train, y_train, epochs=epoch, verbose = False)
                # Evaluate the model on the test data using `evaluate`
                results = model.evaluate(x_test, y_test)
                dic['n_hidden_neurons'].append(hidden_neurons)
                dic['dropout_rate'].append(dropout)
                dic['optimizers'].append(optimizer)
                dic['epochs'].append(epoch)
                dic['results'].append(results)

313/313 [==============================] - 1s 4ms/step - loss: 0.0495 - accuracy: 0.9842


In [ ]:
import pandas as pd

df = pd.DataFrame(dic)
df['accuracy'] = df['results'].apply(lambda x: x[1])
df.sort_values('accuracy', ascending = False)

,n_hidden_neurons,dropout_rate,optimizers,epochs,results,accuracy
13,1024,0.5,adam,300,"[0.4066411554813385, 0.9861999750137329]",0.9862
9,1024,0.1,adam,300,"[0.542689323425293, 0.9846000075340271]",0.9846
15,1024,0.5,SGD,300,"[0.04953382909297943, 0.9842000007629395]",0.9842
7,256,0.5,SGD,300,"[0.05381940305233002, 0.9836000204086304]",0.9836
11,1024,0.1,SGD,300,"[0.05705003812909126, 0.9833999872207642]",0.9834
3,256,0.1,SGD,300,"[0.05819099023938179, 0.983299970626831]",0.9833
12,1024,0.5,adam,10,"[0.06792451441287994, 0.983299970626831]",0.9833
0,256,0.1,adam,10,"[0.0650031566619873, 0.9824000000953674]",0.9824
5,256,0.5,adam,300,"[0.23095504939556122, 0.9819999933242798]",0.9820
4,256,0.5,adam,10,"[0.06832537800073624, 0.9818000197410583]",0.9818


**O que caracteriza a busca em grade?**

É uma operação simples de busca não informado, ou seja, que não aprende com as iterações anteriores, essa operação busca testar todas as combinações de hiperparâmetros e escolher a que tem melhor performance. Esse método, se executado completamente, encontra a melhor arquitetura possível, porém é caro computacionalmente. Nesse caso, como temos poucos hiperparâmetros se torna mais viável utilizar a busca em grade.

Podemos observar que com os resultados obtidos através da busca em grade que obtemos um ganho de mais de 0.03 pontos do melhor resultado para o pior, além disso diferentes combinações de hiperparâmetros produziram resultados muito diferentes, por exemplo, a melhor configuração encontrada utiliza **1024 neurônios**, **dropout de 0.5**, **ADAM** como otimizador e **300 épocas de treinamento**, podemos encontrar esses hiperparâmentros individuais entre as 6 piores combinações, mostrando que a combinação entre bons hiperparâmetros é essencial para uma boa performance do modelo. Contudo vale destacar que ao utilizarmos o otimizador ADAM obtemos bons resultados independente dos hiperparâmetros, com uma variação de, aproximadamente, 0.005 do pior modelo que utiliza ADAM para o melhor modelo (que também utiliza ADAM), já para o SGD a diferença é bem maior indo de 0.9750 para 0.9842 (≅0.02) no melhor modelo usando o SGD, nos mostrando que o impacto da otimização dos hiperparâmetros utilizando SGD como otimizador é bem maior que no ADAM neste caso.
